# Homework 3

---

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error

## 1).
Split the data, stratifying on the outcome variable, survived. You should choose the proportions to split the data into. Verify that the training and testing data sets have the appropriate number of observations. Take a look at the training data and note any potential issues, such as missing data.

Why is it a good idea to use stratified sampling for this data?

It is good to use stratified sampling for the survived column of this data because the amount of the passengers who did not survive is much larger than those who did survive. Thus, by stratifying the data we ensure both the testing and training data maintains essentially the same proportion of survivors to non-survivors, preventing possible bias when training the model. 

In [22]:
# Set a seed so we can reproduce our findings
np.random.seed(100)

# Import the data and change the survived and pclass columns to category
titanic = pd.read_csv('titanic.csv')
titanic['survived'] = titanic['survived'].astype('category')
titanic['survived'] = titanic['survived'].cat.reorder_categories(['Yes', 'No'])
titanic['pclass'] = titanic['pclass'].astype('category')

# Now split the data into training and testing
trainData, testData = train_test_split(titanic, test_size=0.2, stratify=titanic['survived'])

# Verify size of the training and testing data
print(f'Length of total dataset: {len(titanic)}')
print(f'Length of training dataset: {len(trainData)}')
print(f'Length of testing dataset: {len(testData)}')

# Check distribution of the survived column to verify stratification worked
print(f'\nDistribution of training dataset: {trainData["survived"].value_counts(normalize=True)}')
print(f'\nDistribution of testing dataset: {testData["survived"].value_counts(normalize=True)}')

Length of total dataset: 891
Length of training dataset: 712
Length of testing dataset: 179

Distribution of training dataset: survived
No     0.616573
Yes    0.383427
Name: proportion, dtype: float64

Distribution of testing dataset: survived
No     0.614525
Yes    0.385475
Name: proportion, dtype: float64
